In [1]:
cd Downloads/noaa-right-whale-recognition/

C:\Users\reddy\Downloads\noaa-right-whale-recognition


Importing libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import torch
import numpy as np

Reading the csv file

In [3]:
data_label = pd.read_csv('train.csv')
data_label.head()

,Image,whaleID
0,w_7812.jpg,whale_48813
1,w_4598.jpg,whale_09913
2,w_3828.jpg,whale_45062
3,w_8734.jpg,whale_74162
4,w_3251.jpg,whale_99558


In [4]:
data_label.sort_values('Image', inplace=True, ignore_index=True)
data_label.head()

,Image,whaleID
0,w_1.jpg,whale_72820
1,w_100.jpg,whale_66711
2,w_1000.jpg,whale_64496
3,w_1003.jpg,whale_48490
4,w_1004.jpg,whale_70138


Number of images

In [5]:
all_files = os.listdir('imgs/')
print(len(all_files))

11469


Number of classes for classification

In [13]:
unique = data_label['whaleID'].value_counts()
print(unique.shape)

(447,)


In [26]:
Y_train = pd.get_dummies(data_label['whaleID'])

In [27]:
Y_train

,whale_00195,whale_00442,whale_02411,whale_02608,whale_02839,...,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4539,0,0,0,0,0,...,0,0,0,0,0
4540,0,0,0,0,0,...,0,0,0,0,0
4541,0,0,0,0,0,...,0,0,0,0,0
4542,0,0,0,0,0,...,0,0,0,0,0


In [6]:
columns = Y_train.columns

In [11]:
columns

Index(['whale_00195', 'whale_00442', 'whale_02411', 'whale_02608',
       'whale_02839', 'whale_03103', 'whale_03227', 'whale_03623',
       'whale_03728', 'whale_03935',
       ...
       'whale_98618', 'whale_98633', 'whale_98645', 'whale_98746',
       'whale_98939', 'whale_98996', 'whale_99243', 'whale_99326',
       'whale_99558', 'whale_99573'],
      dtype='object', length=447)

Extracting the coordinates from the json file to crop the head of the whales

In [12]:
bonnetheads = pd.read_json('points1.json')
blowholes = pd.read_json('points2.json')

heads_annotations = bonnetheads['annotations'].tolist()
holes_annotations = blowholes['annotations'].tolist()
heads_annotations = [[element[0]['x'], element[0]['y']] for element in heads_annotations]
holes_annotations = [[element[0]['x'], element[0]['y']] for element in holes_annotations]

print('Bonnethead Annotations \n', heads_annotations[:5], '\n')
print('Blowholes Annotations \n', holes_annotations[:5])

Bonnethead Annotations 
 [[1361.8785425101214, 864.3886639676114], [1242.1615154536391, 573.431704885344], [1633.4995014955134, 908.6340977068793], [2431.8723828514453, 784.0797607178464], [3972.849451645065, 1567.77666999003]] 

Blowholes Annotations 
 [[1573.3117408906883, 903.7732793522267], [1289.8105682951145, 384.4785643070787], [1615.1226321036888, 1057.6909272183448], [2209.3080757726816, 939.2622133599201], [3583.489531405783, 1550.5483549351945]]


In [29]:
def extract_training_head_crops(bonnetheads, blowholes, image_directory, filenames):
    # Creating a numpy array of zeros in the required shape for neural network
    features_data = np.zeros(128*256*3).reshape(1,128,256,3)
    
    list_of_files = filenames
    
    # Looping over each file for which the whale ID is to be predicted
    for idx, file in enumerate(list_of_files):
        full_path = image_directory
        image_file_name = os.path.join(full_path, file)
        
        if ".jpg" in image_file_name:

            img_OpenCV = cv2.imread(image_file_name)
            if img_OpenCV is not None:
                print("---------")
                print(idx+1, '/%s'%len(list_of_files))
                print("---------")      

                x1 = int(bonnetheads[idx][0])
                x2 = int(blowholes[idx][0])
                y1 = int(bonnetheads[idx][1])
                y2 = int(blowholes[idx][1])

                ystart = min(y1,y2)-200
                yend = max(y1,y2)+200

                xstart = min(x1,x2)-200
                xend = max(x1,x2)+200

                if ystart<0:
                    ystart=0
                if yend>img_OpenCV.shape[0]:
                    yend = img_OpenCV.shape[0]-1

                if xstart<0:
                    xstart=0
                if xend>img_OpenCV.shape[1]:
                    xend = img_OpenCV.shape[1]-1


                cropped = img_OpenCV[ystart:yend, xstart:xend]
                if abs(x1-x2) < 200:
                    cropped = cv2.rotate(cropped, cv2.ROTATE_90_CLOCKWISE)
                cropped = cv2.resize(cropped, (256,128), interpolation=cv2.INTER_AREA)
                cropped = cropped.reshape(1, 128, 256, 3)
                features_data = np.append(features_data, cropped, axis = 0)
            
    #cleanup - removing first entry added at the top of the function        
    features_data = np.delete(features_data, 0, axis=0)
    return features_data

In [30]:
training_cropped = extract_training_head_crops(heads_annotations, holes_annotations, 'C:/Users/reddy/Downloads/noaa-right-whale-recognition/imgs/imgs', data_label['Image'].tolist())

---------
1 /4544
---------
---------
2 /4544
---------
---------
3 /4544
---------
---------
4 /4544
---------
---------
5 /4544
---------
---------
6 /4544
---------
---------
7 /4544
---------
---------
8 /4544
---------
---------
9 /4544
---------
---------
10 /4544
---------
---------
11 /4544
---------
---------
12 /4544
---------
---------
13 /4544
---------
---------
14 /4544
---------
---------
15 /4544
---------
---------
16 /4544
---------
---------
17 /4544
---------
---------
18 /4544
---------
---------
19 /4544
---------
---------
20 /4544
---------
---------
21 /4544
---------
---------
22 /4544
---------
---------
23 /4544
---------
---------
24 /4544
---------
---------
25 /4544
---------
---------
26 /4544
---------
---------
27 /4544
---------
---------
28 /4544
---------
---------
29 /4544
---------
---------
30 /4544
---------
---------
31 /4544
---------
---------
32 /4544
---------
---------
33 /4544
---------
---------
34 /4544
---------
---------
35 /4544
----

In [32]:
training_cropped.shape

(4544, 128, 256, 3)

In [33]:
np.save('train_crop.npy',training_cropped)

Training the cropped images using Resnet 50 pretrained model

In [8]:
import tensorflow as tf

In [10]:
training_cropped = np.load('train_crop.npy')

In [11]:
Y_train = np.array(Y_train)

In [12]:
base_model = tf.keras.applications.resnet50.ResNet50(include_top=False,
    weights='imagenet',
   
    input_shape=(128,256,3),
    pooling=None,
    classes=447,
)

In [13]:
base_model.trainable = False

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()


In [15]:
prediction_layer = tf.keras.layers.Dense(447)


In [16]:
inputs = tf.keras.Input(shape=(128,256, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
#x = flatten_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [17]:
lr = 0.001
model.compile(tf.keras.optimizers.Adam(learning_rate = lr),loss =tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [18]:
model.fit(training_cropped,Y_train,batch_size = 20,validation_split = 0.2,epochs = 80)

Epoch 1/80
182/182 [==============================] - 23s 86ms/step - loss: 7.1248 - accuracy: 0.0149 - val_loss: 6.3887 - val_accuracy: 0.0330
Epoch 2/80
182/182 [==============================] - 13s 74ms/step - loss: 4.4073 - accuracy: 0.1565 - val_loss: 6.3909 - val_accuracy: 0.0539
Epoch 3/80
182/182 [==============================] - 14s 75ms/step - loss: 3.0251 - accuracy: 0.3585 - val_loss: 6.4985 - val_accuracy: 0.0649
Epoch 4/80
182/182 [==============================] - 14s 74ms/step - loss: 2.1607 - accuracy: 0.5354 - val_loss: 6.5044 - val_accuracy: 0.0803
Epoch 5/80
182/182 [==============================] - 14s 74ms/step - loss: 1.5768 - accuracy: 0.6828 - val_loss: 6.5910 - val_accuracy: 0.0924
Epoch 6/80
182/182 [==============================] - 14s 74ms/step - loss: 1.1862 - accuracy: 0.7791 - val_loss: 6.6442 - val_accuracy: 0.1001
Epoch 7/80
182/182 [==============================] - 14s 75ms/step - loss: 0.9346 - accuracy: 0.8371 - val_loss: 6.8204 - val_accuracy:

In [19]:
tf.keras.models.save_model(model,'resnet')

INFO:tensorflow:Assets written to: resnet\assets


INFO:tensorflow:Assets written to: resnet\assets


PREDICTING THE IMAGES

In [6]:
prediction_files = [file for file in all_files if file not in data_label['Image'].tolist()]
print(len(prediction_files))

6925


In [7]:
import torch

Loading the object detection model trained using YOLOv5

In [8]:
model = torch.hub.load('ultralytics/yolov5','custom' ,path = 'best.pt',force_reload = True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\reddy/.cache\torch\hub\master.zip
YOLOv5  2022-9-4 Python-3.9.12 torch-1.10.1 CUDA:0 (NVIDIA GeForce GTX 1650 Ti, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [9]:
crop = []
for real_img in prediction_files:
        try:
            img = cv2.imread('imgs/'+real_img)
            results = model('imgs/'+real_img)
            aa = results.pred
            xmin = int(aa[0][0][0])
            ymin = int(aa[0][0][1])
            xmax = int(aa[0][0][2])
            ymax = int(aa[0][0][3])
            crop_img = img[ymin:ymax,xmin:xmax]
            resize = cv2.resize(crop_img, (256,128), interpolation=cv2.INTER_AREA)
            cv2.imwrite('crop/'+real_img,resize)
        except:
            crop.append(real_img)
            
            
    
    

In [11]:
crop

['w_10045.jpg',
 'w_10142.jpg',
 'w_10156.jpg',
 'w_10195.jpg',
 'w_1027.jpg',
 'w_10285.jpg',
 'w_10293.jpg',
 'w_10325.jpg',
 'w_10338.jpg',
 'w_10357.jpg',
 'w_10402.jpg',
 'w_10423.jpg',
 'w_10459.jpg',
 'w_10463.jpg',
 'w_10515.jpg',
 'w_10561.jpg',
 'w_10613.jpg',
 'w_10671.jpg',
 'w_10742.jpg',
 'w_10860.jpg',
 'w_10884.jpg',
 'w_10920.jpg',
 'w_10924.jpg',
 'w_10929.jpg',
 'w_10962.jpg',
 'w_11079.jpg',
 'w_11086.jpg',
 'w_11158.jpg',
 'w_11199.jpg',
 'w_11227.jpg',
 'w_11297.jpg',
 'w_11354.jpg',
 'w_11439.jpg',
 'w_1147.jpg',
 'w_1189.jpg',
 'w_1207.jpg',
 'w_1262.jpg',
 'w_1494.jpg',
 'w_1709.jpg',
 'w_1875.jpg',
 'w_2098.jpg',
 'w_2509.jpg',
 'w_3026.jpg',
 'w_3190.jpg',
 'w_3221.jpg',
 'w_3412.jpg',
 'w_362.jpg',
 'w_3633.jpg',
 'w_3726.jpg',
 'w_3791.jpg',
 'w_3883.jpg',
 'w_389.jpg',
 'w_4311.jpg',
 'w_4387.jpg',
 'w_4639.jpg',
 'w_4694.jpg',
 'w_4867.jpg',
 'w_4896.jpg',
 'w_503.jpg',
 'w_5048.jpg',
 'w_5161.jpg',
 'w_5197.jpg',
 'w_5204.jpg',
 'w_5278.jpg',
 'w_5341.jp

In [13]:
for i in crop:
    img = cv2.imread('imgs/'+i)
    resize = cv2.resize(img, (256,128), interpolation=cv2.INTER_AREA)
    cv2.imwrite('crop/'+i,resize)

In [15]:
test = np.array(test)

In [16]:
test.shape

(6925, 128, 256, 3)

In [18]:
from tensorflow import keras

In [20]:
model = keras.models.load_model('resnet')


In [21]:
predictions = model.predict(test,batch_size = 20)

347/347 [==============================] - 22s 56ms/step


In [22]:
results = np.zeros(447).reshape(1,447)
for pred in predictions:
    index = np.argmax(pred)
    arr = np.zeros(447)
    arr[index] = 1.0
    arr = arr.reshape(1,447)
    results = np.append(results, arr, axis = 0)

In [23]:
results = np.delete(results, 0, axis=0)
results[0]

array([          0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,           0,
                 0,           0,           

In [28]:
columns = Y_train.columns

In [29]:
results_df = pd.DataFrame(results, columns = columns)

In [30]:
fil= pd.DataFrame(prediction_files, columns=['Image'])

In [31]:
d2f = pd.concat((fil, results_df), axis=1, join='inner')

In [32]:
sub = pd.read_csv('sample_submission.csv')

In [33]:
sub = sub['Image'].tolist()

In [34]:
sorterIndex = dict(zip(sub,range(len(sub))))

In [35]:
d2f['temp_rank'] = d2f['Image'].map(sorterIndex)

In [36]:
d2f.sort_values(['temp_rank'], ascending = True, inplace = True, ignore_index=True)
d2f.drop('temp_rank', 1, inplace = True)

C:\Users\reddy\AppData\Local\Temp\ipykernel_10736\3286886341.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  d2f.drop('temp_rank', 1, inplace = True)


In [37]:
d2f.to_csv('submission.csv', index=False)

,Image,whale_00195,whale_00442,whale_02411,whale_02608,...,whale_98996,whale_99243,whale_99326,whale_99558,whale_99573
0,w_1947.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
1,w_11096.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
2,w_10973.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,w_10442.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
4,w_10606.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
6920,w_4867.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
6921,w_5230.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
6922,w_5278.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
6923,w_9218.jpg,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
